In [120]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/diploma/data.csv


In [121]:
# импорт библиотек

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling
from pandas_profiling import ProfileReport
import os
import sys
import zipfile
import PIL
from PIL import ImageOps, ImageFilter
import re
import ast
from datetime import time, date, datetime
import random

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-paper')

#from optuna import *

from sklearn import preprocessing
#from sklearn.feature_extraction import DictVectorizer
#from sklearn.feature_selection import SelectKBest, chi2, f_classif, f_regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler, RobustScaler
#from sklearn.impute import KNNImputer

#from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
#from sklearn.linear_model import Ridge
#from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score
from sklearn import metrics
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import VotingRegressor
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor, Pool, cv

#image augmentation
#import albumentations as A
#from ImageDataAugmentor.image_data_augmentor import *

# keras
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#from tensorflow.keras.applications.efficientnet import EfficientNetB3
#from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import MeanAbsolutePercentageError

#from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, DataCollatorWithPadding

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', 550) # больше колонок

os.environ['PYTHONHASHSEED']=str(42)

In [122]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
ITERATIONS = 200

In [123]:
DIR_DATA  = '../input/diploma/'

In [124]:
data_d = pd.read_csv(DIR_DATA+'data.csv')

In [125]:
data_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [126]:
data_d.head(10)

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"
5,for sale,NaN,townhouse,1624 S Newkirk St,NaN,"{'atAGlanceFacts': [{'factValue': '1920', 'fac...",NaN,Philadelphia,"[{'rating': [], 'data': {'Distance': [], 'Grad...",897 sqft,19145,2 Beds,PA,2.0,NaN,NaN,PAPH847006,"$209,000"
6,Active,NaN,Florida,552 Casanova Ct,NaN,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",NaN,POINCIANA,"[{'rating': ['3', '3', '1', 'NR'], 'data': {'D...","1,507",34759,NaN,FL,One,NaN,NaN,S5026943,"181,500"
7,Active,NaN,NaN,6094 Mingle Dr,NaN,"{'atAGlanceFacts': [{'factValue': '1976', 'fac...",NaN,Memphis,"[{'rating': ['4', '2', '2'], 'data': {'Distanc...",NaN,38115,NaN,TN,NaN,NaN,NaN,10063506,"68,000"
8,Active,NaN,Single Family Home,11182 Owl Ave,2,"{'atAGlanceFacts': [{'factValue': '1970', 'fac...",NaN,Mason City,"[{'rating': ['2', '2', '4', '7', '4', 'NR'], '...",3588,50401,3,IA,NaN,NaN,NaN,190988,"$244,900"
9,NaN,NaN,Single Family,8612 Cedar Plains Ln,3,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Houston,"[{'rating': ['4/10', '3/10', '2/10'], 'data': ...","1,930",77080,3,TX,2.0,NaN,NaN,73968331,"$311,995"


In [127]:
data_d_small = data_d.loc[data_d.index<10]

In [128]:
data_d_small.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   status        9 non-null      object
 1   private pool  0 non-null      object
 2   propertyType  9 non-null      object
 3   street        10 non-null     object
 4   baths         6 non-null      object
 5   homeFacts     10 non-null     object
 6   fireplace     3 non-null      object
 7   city          10 non-null     object
 8   schools       10 non-null     object
 9   sqft          8 non-null      object
 10  zipcode       10 non-null     object
 11  beds          7 non-null      object
 12  state         10 non-null     object
 13  stories       6 non-null      object
 14  mls-id        0 non-null      object
 15  PrivatePool   1 non-null      object
 16  MlsId         10 non-null     object
 17  target        10 non-null     object
dtypes: object(18)
memory usage: 1.5+ KB


In [129]:
data_d.schools = data_d.schools.apply(lambda x: eval(x[1:-1]))

In [130]:
t = data_d['schools']
t

0         {'rating': ['4', '4', '7', 'NR', '4', '7', 'NR...
1         {'rating': ['4/10', 'None/10', '4/10'], 'data'...
2         {'rating': ['8/10', '4/10', '8/10'], 'data': {...
3         {'rating': ['9/10', '9/10', '10/10', '9/10'], ...
4         {'rating': ['4/10', '5/10', '5/10'], 'data': {...
                                ...                        
377180    {'rating': ['10/10', '5/10'], 'data': {'Distan...
377181    {'rating': ['1/10', '5/10', '7/10'], 'data': {...
377182    {'rating': ['5/10', '4/10'], 'data': {'Distanc...
377183    {'rating': ['NA', 'NA', 'NA'], 'data': {'Dista...
377184    {'rating': ['5/10', '4/10', '3/10'], 'data': {...
Name: schools, Length: 377185, dtype: object

In [131]:
t_lst = t.tolist()

In [132]:
t5_lst = []
for i in t_lst:
    i1 = dict([('rating', i.get('rating')), ('Distance', (i.get('data')).get('Distance')), ('Grades', (i.get('data')).get('Grades')),('name',i.get('name'))])
    t5_lst.append(i1)

In [133]:
df_lst = pd.DataFrame(t5_lst,dtype=str)

In [134]:
df_lst = pd.DataFrame(t5_lst,dtype=str)

In [135]:
data_d.head(3)

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"{'rating': ['4', '4', '7', 'NR', '4', '7', 'NR...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"{'rating': ['4/10', 'None/10', '4/10'], 'data'...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"{'rating': ['8/10', '4/10', '8/10'], 'data': {...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"


In [136]:
data_d = data_d.join(df_lst)

In [137]:
data_d.head(3)

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target,rating,Distance,Grades,name
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"{'rating': ['4', '4', '7', 'NR', '4', '7', 'NR...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000","['4', '4', '7', 'NR', '4', '7', 'NR', 'NR']","['2.7 mi', '3.6 mi', '5.1 mi', '4.0 mi', '10.5...","['3–5', '6–8', '9–12', 'PK–2', '6–8', '9–12', ...","['Southern Pines Elementary School', 'Southern..."
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"{'rating': ['4/10', 'None/10', '4/10'], 'data'...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000","['4/10', 'None/10', '4/10']","['1.65mi', '1.32mi', '1.01mi']","['9-12', '3-8', 'PK-8']","['East Valley High School&Extension', 'Eastval..."
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"{'rating': ['8/10', '4/10', '8/10'], 'data': {...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000","['8/10', '4/10', '8/10']","['1.19mi', '2.06mi', '2.63mi']","['6-8', 'K-5', '9-12']","['Paul Revere Middle School', 'Brentwood Scien..."


In [138]:
data_d['type']=data_d['rating'].apply(lambda x: type(x))

In [139]:
#def school_rating (lst_r):
#    lst_clean = []
#    if lst_r == []: return [0, 0, 0]
#    elif isinstance(lst_r, list) == False:
#        return [0, 0, 0]
#    else:
#        for p in lst_r:
#            
#            if p.find('/') == -1:
#                if p.isdigit():
#                    lst_clean.append(int(p))
#                else: lst_clean.append(0)
#            else:
#        
#            
#                    if p.find('/'):
#                        num = p[:p.find('/')]
#                        
#                        if not num.isdigit():
#                            lst_clean.append(0)
#                        else: lst_clean.append(int(num))
#
#    return lst_clean

In [140]:
def school_rating1 (lst_r):
    lst_clean = []
    for p in lst_r:
        if p.find('/') == -1:
            if p.isdigit():
                lst_clean.append(int(p))
            else: lst_clean.append(0)
        else:
            num = p[:p.find('/')]
            if num.isdigit():
                lst_clean.append(int(num))
                
            else: lst_clean.append(0)

    return lst_clean

In [141]:
shd = ['10/10', '4', '7', 'NR', '4', '7', 'NR', 'NR']

In [142]:
school_rating(shd)

[10, 4, 7, 0, 4, 7, 0, 0]

In [143]:
data_d.rating = data_d.rating.apply(lambda x: school_rating1(x))

In [144]:
data_d.head(5)

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target,rating,Distance,Grades,name,type
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"{'rating': ['4', '4', '7', 'NR', '4', '7', 'NR...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000","[0, 0, 4, 0, 0, 0, 0, 4, 0, 0, 0, 0, 7, 0, 0, ...","['2.7 mi', '3.6 mi', '5.1 mi', '4.0 mi', '10.5...","['3–5', '6–8', '9–12', 'PK–2', '6–8', '9–12', ...","['Southern Pines Elementary School', 'Southern...",<class 'str'>
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"{'rating': ['4/10', 'None/10', '4/10'], 'data'...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000","[0, 0, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","['1.65mi', '1.32mi', '1.01mi']","['9-12', '3-8', 'PK-8']","['East Valley High School&Extension', 'Eastval...",<class 'str'>
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"{'rating': ['8/10', '4/10', '8/10'], 'data': {...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000","[0, 0, 8, 0, 1, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, ...","['1.19mi', '2.06mi', '2.63mi']","['6-8', 'K-5', '9-12']","['Paul Revere Middle School', 'Brentwood Scien...",<class 'str'>
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"{'rating': ['9/10', '9/10', '10/10', '9/10'], ...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000","[0, 0, 9, 0, 1, 0, 0, 0, 0, 0, 9, 0, 1, 0, 0, ...","['1.05mi', '0.1mi', '1.05mi', '0.81mi']","['5-6', 'PK-4', '7-8', '9-12']","['Mcculloch Intermediate School', 'Bradfield E...",<class 'str'>
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"{'rating': ['4/10', '5/10', '5/10'], 'data': {...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000","[0, 0, 4, 0, 1, 0, 0, 0, 0, 0, 5, 0, 1, 0, 0, ...","['5.96mi', '3.25mi', '3.03mi']","['7-8', '9-12', 'PK-6']","['Southwest Middle School', 'Bayside High Scho...",<class 'str'>
